# Spark Coding Assignment

In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
    .appName("question1")\
    .master("local")\
    .getOrCreate()

In [4]:
df=spark.read.csv("startup.csv",inferSchema=True,header=True)

In [5]:
df1=spark.read.parquet("consumerInternet.parquet",inferSchema=True,header=True)

# Combine these 2 files in Spark into a single entity 

In [6]:
df3=df.unionAll(df1)

In [7]:
df3.printSchema()

root
 |-- Sr_No: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Startup_Name: string (nullable = true)
 |-- Industry_Vertical: string (nullable = true)
 |-- SubVertical: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Investors_Name: string (nullable = true)
 |-- InvestmentnType: string (nullable = true)
 |-- Amount_in_USD: string (nullable = true)
 |-- Remarks: string (nullable = true)



# How many startups are there in Pune City?


In [8]:
df3.createOrReplaceTempView("df3")
sqlWay1 = spark.sql("""
SELECT count(Startup_Name) as Pune_Startup
FROM df3
WHERE City='Pune'
""")
sqlWay1.show()

+------------+
|Pune_Startup|
+------------+
|         105|
+------------+



# How many startups in Pune got their Seed/ Angel Funding?

In [9]:
sqlway2=spark.sql("""
SELECT COUNT(Startup_Name) as Pune_Startup
FROM df3 
WHERE InvestmentnType like 'Seed%/%' 
AND City = 'Pune'""")
sqlway2.show()

+------------+
|Pune_Startup|
+------------+
|           6|
+------------+



# What is the total amount raised by startups in Pune City? Hint - use regex_replace to get rid of null


In [10]:
sqlway3=spark.sql("""
SELECT SUM(regexp_replace(Amount_in_USD, 'N/A', '00')) AS Amount 
FROM df3
WHERE City = 'Pune'""")
sqlway3.show()

+------+
|Amount|
+------+
|   0.0|
+------+



# What are the top 5 Industry_Vertical which has the highest number of startups in India?


In [11]:
sqlway4=spark.sql("""
SELECT Industry_Vertical, COUNT(Startup_Name) 
FROM df3 
GROUP BY Industry_Vertical 
ORDER BY COUNT(Startup_Name) DESC LIMIT 5""")
sqlway4.show()

+-----------------+-------------------+
|Industry_Vertical|count(Startup_Name)|
+-----------------+-------------------+
|Consumer Internet|                941|
|       Technology|                478|
|        eCommerce|                186|
|              nan|                171|
|       Healthcare|                 70|
+-----------------+-------------------+



# Find the top Investor(by amount) of each year.


In [15]:
sqlway5=spark.sql("""
SELECT Investors_Name,
(SELECT DISTINCT(Year("Date")) from DF3) as Year, 
Amount_in_USD
FROM df3
ORDER BY Amount_in_USD LIMIT 5""")
sqlway5.show()

+--------------------+----+--------------+
|      Investors_Name|Year| Amount_in_USD|
+--------------------+----+--------------+
|Vijay Shekhar Sharma|null|1,00,00,00,000|
|     Eduardo Saverin|null|   1,00,00,000|
|HDFC Life, HDFC A...|null|   1,00,00,000|
|Accel Partners, T...|null|   1,00,00,000|
|Sabre Partners, P...|null|   1,00,00,000|
+--------------------+----+--------------+

